In [95]:
import torch

device = torch.device("cuda:7")

In [96]:
from spuco.utils import set_seed

set_seed(0)

In [97]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_LARGE

trainset = SpuCoMNIST(
    root="../data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.95,
    classes=classes,
    split="train",
    label_noise=0
)
trainset.initialize()

testset = SpuCoMNIST(
    root="../data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="test"
)
testset.initialize()

100%|██████████| 10000/10000 [00:02<00:00, 4196.09it/s]


In [98]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]

valset = SpuCoMNIST(
    root="../data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="val"
)
valset.initialize()

 61%|██████▏   | 7362/11996 [00:01<00:00, 4715.01it/s]

100%|██████████| 11996/11996 [00:02<00:00, 4752.68it/s]


In [99]:
T.ToPILImage()(trainset[10732][0]).resize((28,28))

## Implement the Dataloader

In [100]:
from torch.utils.data import DataLoader, Dataset

class EnDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        x, y = self.dataset[idx]
        return x, y, idx

train_dataloader = DataLoader(EnDataset(trainset), batch_size=128, shuffle=True, num_workers=4)
val_dataloader = DataLoader(EnDataset(valset), batch_size=128, shuffle=False, num_workers=4)
test_dataloader = DataLoader(EnDataset(testset), batch_size=128, shuffle=False, num_workers=4)

# Setting the Hyperparameter

In [101]:
epochs = 10

# Implement the robust_model and Optimizer

In [102]:
from spuco.models import model_factory
from torch.optim import Adam
model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(device)
optimizer = Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)

# Trainning Process

In [103]:
from tqdm import tqdm
import numpy as np

# Calculate the forgetting score during training
prev_acc = np.zeros(len(trainset))
forgetting = np.zeros(len(trainset))

for epoch in range(epochs):
    model.train()
    with tqdm(total=len(train_dataloader), dynamic_ncols=True) as t:
        t.set_description(f"Epoch {epoch}/{epochs}")
        for x, y, idx in train_dataloader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            y_hat = model(x)
            loss = torch.nn.functional.cross_entropy(y_hat, y)
            acc = (y_hat.argmax(dim=1) == y).float()

            # Calculate forgetting
            for batch_idx, i in enumerate(idx):
                if prev_acc[i] > acc[batch_idx]:
                    forgetting[i] = forgetting[i]+1
            
            prev_acc[idx] = acc.cpu().numpy()
            acc = acc.mean()

            loss.backward()
            optimizer.step()

            t.set_postfix({
                "loss": loss.item(),
                "acc": acc.item()
            })
            t.update()

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for x, y, idx in val_dataloader:
            x, y = x.to(device), y.to(device)
            y_hat = model(x)
            correct += (y_hat.argmax(dim=1) == y).sum().item()
            total += y.size(0)
    print(f"Validation accuracy: {correct/total}")

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for x, y, idx in test_dataloader:
        x, y = x.to(device), y.to(device)
        y_hat = model(x)
        correct += (y_hat.argmax(dim=1) == y).sum().item()
        total += y.size(0)
print(f"Test accuracy: {correct/total}")
        

Epoch 0/10: 100%|██████████| 376/376 [00:07<00:00, 50.02it/s, loss=0.0412, acc=1]    


Validation accuracy: 0.2031510503501167


Epoch 1/10: 100%|██████████| 376/376 [00:07<00:00, 53.47it/s, loss=0.00854, acc=1]   


Validation accuracy: 0.3995498499499833


Epoch 2/10: 100%|██████████| 376/376 [00:06<00:00, 54.94it/s, loss=0.00629, acc=1]   


...

Validation accuracy: 0.8738746248749584


Epoch 8/10: 100%|██████████| 376/376 [00:06<00:00, 54.73it/s, loss=0.000555, acc=1]   


Validation accuracy: 0.8537012337445815


Epoch 9/10: 100%|██████████| 376/376 [00:06<00:00, 54.56it/s, loss=6.11e-5, acc=1]    


Validation accuracy: 0.9023007669223074
Test accuracy: 0.9099


# Calculating wrong index

In [104]:
model.eval()
wrong_idx = []
for x, y, idx in train_dataloader:
    x, y, idx = x.to(device), y.to(device), idx.to(device)
    y_hat = model(x)
    wrong_batch_idx = (y_hat.argmax(dim=1) != y).nonzero()
    wrong_idx.append(idx[wrong_batch_idx].cpu().numpy())

wrong_idx = np.concatenate(wrong_idx)


In [113]:
print(len(np.nonzero(forgetting[wrong_idx])[0]))
print(len(wrong_idx))
print(forgetting[wrong_idx].mean())
print(forgetting[wrong_idx].max())

95
189
0.7037037037037037
3.0


# JTT, Choose from 1,2 to run

In [107]:
upsample_factor = 10

## 1. Original JTT

In [ ]:
from torch.utils.data import Subset
upsample_idx = np.concat((np.arange(len(trainset)), np.repeat(wrong_idx, upsample_factor)))
upsample_trainset = Subset(trainset, upsample_idx)

array([    0,     1,     2, ..., 47953, 47953, 47953])

## 2. Based on Forgrtting Scores

In [114]:
wrong_sample_fgscore = forgetting[wrong_idx]
total_repeats = upsample_factor * len(wrong_idx)
initial_repeats = wrong_sample_fgscore / wrong_sample_fgscore.sum() * total_repeats
repeats = np.round(initial_repeats).astype(int)

while repeats.sum() != total_repeats:
    diff = total_repeats - repeats.sum()
    if diff > 0:
        repeats[np.argmax(initial_repeats - repeats)] += 1
    else:
        repeats[np.argmax(repeats - initial_repeats)] -= 1

repeats = repeats.squeeze()
upsample_idx = np.concat((np.arange(len(trainset)), np.repeat(wrong_idx, repeats)))
upsample_trainset = Subset(trainset, upsample_idx)

# Training

In [115]:
train_dataloader = DataLoader(EnDataset(upsample_trainset), batch_size=128, shuffle=True, num_workers=4)

In [116]:
robust_model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(device)
optimizer = Adam(robust_model.parameters(), lr=1e-3, weight_decay=1e-4)

In [117]:
from tqdm import tqdm
import numpy as np

for epoch in range(epochs):
    robust_model.train()
    with tqdm(total=len(train_dataloader), dynamic_ncols=True) as t:
        t.set_description(f"Epoch {epoch}/{epochs}")
        for x, y, idx in train_dataloader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            y_hat = robust_model(x)
            loss = torch.nn.functional.cross_entropy(y_hat, y)
            acc = (y_hat.argmax(dim=1) == y).float()
            acc = acc.mean()

            loss.backward()
            optimizer.step()

            t.set_postfix({
                "loss": loss.item(),
                "acc": acc.item()
            })
            t.update()

    robust_model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for x, y, idx in val_dataloader:
            x, y = x.to(device), y.to(device)
            y_hat = robust_model(x)
            correct += (y_hat.argmax(dim=1) == y).sum().item()
            total += y.size(0)
    print(f"Validation accuracy: {correct/total}")

robust_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for x, y, idx in test_dataloader:
        x, y = x.to(device), y.to(device)
        y_hat = robust_model(x)
        correct += (y_hat.argmax(dim=1) == y).sum().item()
        total += y.size(0)
print(f"Test accuracy: {correct/total}")

Epoch 0/10: 100%|██████████| 390/390 [00:04<00:00, 90.56it/s, loss=0.132, acc=0.971]  


Validation accuracy: 0.22074024674891632


Epoch 1/10: 100%|██████████| 390/390 [00:04<00:00, 90.31it/s, loss=0.0911, acc=0.971] 


Validation accuracy: 0.41980660220073357


Epoch 2/10: 100%|██████████| 390/390 [00:04<00:00, 90.46it/s, loss=0.0669, acc=0.98]  


...

Validation accuracy: 0.8617872624208069


Epoch 8/10: 100%|██████████| 390/390 [00:04<00:00, 91.90it/s, loss=0.0183, acc=0.99]   


Validation accuracy: 0.8562020673557853


Epoch 9/10: 100%|██████████| 390/390 [00:04<00:00, 91.91it/s, loss=0.0636, acc=0.98]   


Validation accuracy: 0.8746248749583194
Test accuracy: 0.8844
